In [ ]:
# 1. Cài thư viện (Bản GPU)
!pip install ultralytics paddlepaddle-gpu paddleocr==2.7.3

# 2. Fix lỗi xung đột Numpy (Bắt buộc)
!pip install "numpy<2.0.0" --force-reinstall

  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.


  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
# Giải nén lại model
!tar -xvf /content/yolov11-lp-nano-other-default-v1.tar.gz

kaggle/working/
kaggle/
kaggle/working/data.yaml
kaggle/working/runs/segment/predict/
kaggle/working/runs/segment/
kaggle/working/runs/
kaggle/working/runs/segment/predict/Tgmt_0050.jpg
kaggle/working/runs/segment/val/
kaggle/working/runs/segment/val/BoxF1_curve.png
kaggle/working/runs/segment/val/BoxPR_curve.png
kaggle/working/runs/segment/val/BoxP_curve.png
kaggle/working/runs/segment/val/BoxR_curve.png
kaggle/working/runs/segment/val/MaskF1_curve.png
kaggle/working/runs/segment/val/MaskPR_curve.png
kaggle/working/runs/segment/val/MaskP_curve.png
kaggle/working/runs/segment/val/MaskR_curve.png
kaggle/working/runs/segment/val/confusion_matrix.png
kaggle/working/runs/segment/val/confusion_matrix_normalized.png
kaggle/working/runs/segment/val/val_batch0_labels.jpg
kaggle/working/runs/segment/val/val_batch0_pred.jpg
kaggle/working/runs/segment/val/val_batch1_labels.jpg
kaggle/working/runs/segment/val/val_batch1_pred.jpg
kaggle/working/runs/segment/val/val_batch2_labels.jpg
kaggle/working

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from paddleocr import PaddleOCR
import time
import os
import re

# ==========================================
# CẤU HÌNH HỆ THỐNG
# ==========================================
# Đường dẫn model YOLO (Đảm bảo file này tồn tại)
YOLO_PATH = '/content/kaggle/working/runs/segment/yolo11_license_plate/weights/best.pt'
# Tên file video đầu vào (Upload lên Colab)
VIDEO_INPUT = 'input_video0.mp4'
# Tên file video kết quả (Sẽ được tạo ra)
VIDEO_OUTPUT = 'output_video_result.mp4'

# ==========================================
# CÁC HÀM XỬ LÝ LOGIC (HẬU XỬ LÝ)
# ==========================================
def clean_text(text):
    """Làm sạch rác: chỉ giữ lại chữ cái và số, viết hoa"""
    return re.sub(r'[^A-Za-z0-9]', '', text).upper()

def format_plate(text):
    """
    Định dạng lại biển số cho đẹp theo chuẩn Việt Nam:
    - Biển 4 số cũ: 29H1234 -> 29H-1234
    - Biển 5 số mới: 30A12345 -> 30A-123.45
    """
    if len(text) < 4: return text

    # Logic định dạng dựa trên độ dài chuỗi
    if len(text) == 8: # Chuẩn biển 5 số (VD: 30A12345)
        return f"{text[:3]}-{text[3:6]}.{text[6:]}"
    elif len(text) == 7: # Chuẩn biển 4 số (VD: 29H1234)
        return f"{text[:3]}-{text[3:]}"
    elif len(text) == 9: # Trường hợp thừa 1 ký tự (thường là do OCR đọc nhầm dấu chấm thành số)
        # VD: 30A123045 (thừa số 0 ở giữa) -> Cố gắng sửa
        return f"{text[:3]}-{text[3:6]}.{text[7:]}"

    return text # Trả về gốc nếu không khớp rule nào

def process_ocr_result(ocr_result):
    """Xử lý kết quả thô từ PaddleOCR"""
    if ocr_result is None: return ""

    # Sắp xếp các dòng text từ trên xuống dưới (quan trọng cho biển vuông 2 dòng)
    sorted_res = sorted(ocr_result, key=lambda x: x[0][0][1])

    raw_text = ""
    for line in sorted_res:
        text, conf = line[1]
        # Chỉ lấy text nếu độ tin cậy > 50%
        if conf > 0.5:
            raw_text += text

    # Làm sạch và định dạng lại
    cleaned = clean_text(raw_text)
    return format_plate(cleaned)

# ==========================================
# CHƯƠNG TRÌNH CHÍNH (MAIN LOOP)
# ==========================================
def run_video_processing():
    # 1. Kiểm tra file
    if not os.path.exists(YOLO_PATH):
        print("❌ LỖI: Không tìm thấy file model YOLO. Hãy giải nén lại file .tar.gz")
        return
    if not os.path.exists(VIDEO_INPUT):
        print(f"❌ LỖI: Không tìm thấy video '{VIDEO_INPUT}'. Hãy upload video lên Colab.")
        return

    print(">>> Đang khởi tạo hệ thống...")
    print("    [1/3] Load YOLO trên GPU (T4)...")
    yolo = YOLO(YOLO_PATH)

    print("    [2/3] Load PaddleOCR trên CPU (Để tránh xung đột RAM)...")
    # use_gpu=False là CHÌA KHÓA để không bị sập Colab
    ocr = PaddleOCR(use_angle_cls=False, lang='en', show_log=False, use_gpu=False)

    print(f"    [3/3] Đang mở video: {VIDEO_INPUT}")
    cap = cv2.VideoCapture(VIDEO_INPUT)

    # Lấy thông số video
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Khởi tạo Video Writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(VIDEO_OUTPUT, fourcc, fps, (w, h))

    print(f"    --> Độ phân giải: {w}x{h} | FPS: {fps:.1f} | Tổng: {total_frames} frames")
    print("\n>>> BẮT ĐẦU XỬ LÝ (Vui lòng đợi)...")

    frame_idx = 0
    start_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break # Hết video

        frame_idx += 1

        # --- BƯỚC 1: PHÁT HIỆN BIỂN SỐ (YOLO - GPU) ---
        # verbose=False để đỡ spam log
        results = yolo(frame, verbose=False)[0]

        # --- BƯỚC 2: ĐỌC CHỮ (PaddleOCR - CPU) ---
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Crop ảnh biển số
            # Thêm padding nhẹ (2px) để không cắt sát quá
            h_img, w_img, _ = frame.shape
            crop = frame[max(0, y1-2):min(h_img, y2+2), max(0, x1-2):min(w_img, x2+2)]

            # Bỏ qua nếu ảnh quá nhỏ (biển số ở xa tít) để tăng tốc
            if crop.shape[0] < 20 or crop.shape[1] < 20: continue

            try:
                # cls=False tắt chức năng xoay ngược ảnh (giúp nhanh hơn cho video)
                ocr_res = ocr.ocr(crop, cls=False)

                if ocr_res and ocr_res[0]:
                    plate_text = process_ocr_result(ocr_res[0])

                    if len(plate_text) >= 4: # Chỉ vẽ nếu đọc được ít nhất 4 ký tự
                        # Vẽ khung chữ nhật
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                        # Vẽ nền chữ màu đen để dễ đọc
                        (text_w, text_h), _ = cv2.getTextSize(plate_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
                        cv2.rectangle(frame, (x1, y1 - 30), (x1 + text_w, y1), (0, 255, 0), -1)

                        # Viết chữ biển số
                        cv2.putText(frame, plate_text, (x1, y1 - 8),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
            except:
                pass # Bỏ qua lỗi frame xấu

        # Ghi frame đã xử lý vào video đầu ra
        out.write(frame)

        # Log tiến độ mỗi 20 frames
        if frame_idx % 20 == 0:
            elapsed = time.time() - start_time
            fps_real = frame_idx / elapsed
            percent = (frame_idx / total_frames) * 100
            print(f"   Đang xử lý: {frame_idx}/{total_frames} ({percent:.1f}%) - Tốc độ: {fps_real:.1f} FPS")

    # Dọn dẹp
    cap.release()
    out.release()
    print("\n" + "="*40)
    print("✅ HOÀN TẤT! Video đã được lưu.")
    print(f"📂 Tên file kết quả: {VIDEO_OUTPUT}")
    print("👉 Hãy download file này về máy để xem.")
    print("="*40)

# Chạy hàm
if __name__ == "__main__":
    run_video_processing()

>>> Đang khởi tạo hệ thống...
    [1/3] Load YOLO trên GPU (T4)...
    [2/3] Load PaddleOCR trên CPU (Để tránh xung đột RAM)...
download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:16<00:00, 240kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:25<00:00, 395kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:15<00:00, 142kiB/s]


    [3/3] Đang mở video: /content/drive/MyDrive/CV/Bản sao của input_video0.mp4
    --> Độ phân giải: 1920x1080 | FPS: 30.0 | Tổng: 2779 frames

>>> BẮT ĐẦU XỬ LÝ (Vui lòng đợi)...
   Đang xử lý: 20/2779 (0.7%) - Tốc độ: 7.0 FPS
   Đang xử lý: 40/2779 (1.4%) - Tốc độ: 11.1 FPS
   Đang xử lý: 60/2779 (2.2%) - Tốc độ: 14.2 FPS
   Đang xử lý: 80/2779 (2.9%) - Tốc độ: 17.0 FPS
   Đang xử lý: 100/2779 (3.6%) - Tốc độ: 19.2 FPS
   Đang xử lý: 120/2779 (4.3%) - Tốc độ: 21.0 FPS
   Đang xử lý: 140/2779 (5.0%) - Tốc độ: 20.9 FPS
   Đang xử lý: 160/2779 (5.8%) - Tốc độ: 19.3 FPS
   Đang xử lý: 180/2779 (6.5%) - Tốc độ: 18.2 FPS
   Đang xử lý: 200/2779 (7.2%) - Tốc độ: 17.5 FPS
   Đang xử lý: 220/2779 (7.9%) - Tốc độ: 16.7 FPS
   Đang xử lý: 240/2779 (8.6%) - Tốc độ: 16.3 FPS
   Đang xử lý: 260/2779 (9.4%) - Tốc độ: 16.1 FPS
   Đang xử lý: 280/2779 (10.1%) - Tốc độ: 16.5 FPS
   Đang xử lý: 300/2779 (10.8%) - Tốc độ: 17.0 FPS
   Đang xử lý: 320/2779 (11.5%) - Tốc độ: 17.2 FPS
   Đang xử lý: 340/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
